In [ ]:
!pip3 install -U tf2onnx onnxruntime

In [1]:
import tensorflow as tf

In [4]:
from wav2vec2 import Wav2Vec2ForCTC
MODEL_ID = "vasudevgupta/tf-wav2vec2-base-960h"
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

Total number of loaded variables: 213


In [16]:
import soundfile as sf
speech, _ = sf.read("../data/SA2.wav")
speech = tf.expand_dims(tf.constant(speech, dtype=tf.float32), 0)
speech.shape

TensorShape([1, 47104])

In [15]:
outputs = model(speech)
print(outputs.shape)

(1, 146, 32)


In [30]:
import tf2onnx
ONNX_PATH = "onnx-wav2vec2.onnx"

input_signature = (tf.TensorSpec((None, 47104), tf.float32, name="speech"),)
_ = tf2onnx.convert.from_keras(model, input_signature=input_signature, output_path=ONNX_PATH)

In [44]:
import onnxruntime as rt

sess = rt.InferenceSession(ONNX_PATH)
onnx_outputs = sess.run(None, {"speech": speech.numpy()})
onnx_outputs

[array([[[ 13.740026 , -26.97787  , -26.698689 , ...,  -6.6399283,
           -8.961572 ,  -6.4920783],
         [ 13.659376 , -27.625504 , -27.344852 , ...,  -7.2058935,
           -9.226441 ,  -6.604441 ],
         [ 13.97566  , -26.724403 , -26.42656  , ...,  -6.85717  ,
           -8.794083 ,  -6.7176375],
         ...,
         [ 13.886351 , -26.631052 , -26.376377 , ...,  -5.5348268,
           -8.127068 ,  -6.619991 ],
         [ 13.886124 , -26.630331 , -26.37566  , ...,  -5.534939 ,
           -8.126596 ,  -6.6206064],
         [ 13.668737 , -27.154783 , -26.876873 , ...,  -6.8891983,
           -9.146206 ,  -6.184449 ]]], dtype=float32)]

In [35]:
import time

start = time.time()
onnx_outputs = sess.run(None, {"speech": speech.numpy()})
print(onnx_outputs)
print("TIME TAKEN:", time.time() - start)

[array([[[ 13.740026 , -26.97787  , -26.698689 , ...,  -6.6399283,
          -8.961572 ,  -6.4920783],
        [ 13.659376 , -27.625504 , -27.344852 , ...,  -7.2058935,
          -9.226441 ,  -6.604441 ],
        [ 13.97566  , -26.724403 , -26.42656  , ...,  -6.85717  ,
          -8.794083 ,  -6.7176375],
        ...,
        [ 13.886351 , -26.631052 , -26.376377 , ...,  -5.5348268,
          -8.127068 ,  -6.619991 ],
        [ 13.886124 , -26.630331 , -26.37566  , ...,  -5.534939 ,
          -8.126596 ,  -6.6206064],
        [ 13.668737 , -27.154783 , -26.876873 , ...,  -6.8891983,
          -9.146206 ,  -6.184449 ]]], dtype=float32)]
TIME TAKEN: 0.3857080936431885


In [36]:
start = time.time()
outputs = model(speech)
print(outputs)
print("TIME TAKEN:", time.time() - start)

tf.Tensor(
[[[ 13.740425  -26.978495  -26.699314  ...  -6.6403112  -8.962097
    -6.491789 ]
  [ 13.659467  -27.625984  -27.34533   ...  -7.2062693  -9.226593
    -6.604583 ]
  [ 13.976038  -26.726124  -26.428267  ...  -6.8579597  -8.794957
    -6.717271 ]
  ...
  [ 13.886277  -26.630238  -26.375546  ...  -5.534626   -8.126781
    -6.619562 ]
  [ 13.88648   -26.629833  -26.375141  ...  -5.534644   -8.126343
    -6.620388 ]
  [ 13.669438  -27.155434  -26.87751   ...  -6.8898835  -9.146654
    -6.184311 ]]], shape=(1, 146, 32), dtype=float32)
TIME TAKEN: 2.4164552688598633


In [37]:
@tf.function(jit_compile=True)
def forward(speech):
    return model(speech)

In [41]:
start = time.time()
outputs = forward(speech)
print(outputs)
print("TIME TAKEN:", time.time() - start)

tf.Tensor(
[[[ 13.740374  -26.979073  -26.699873  ...  -6.6408443  -8.962109
    -6.4920826]
  [ 13.659096  -27.625633  -27.34498   ...  -7.205656   -9.226397
    -6.6042814]
  [ 13.975779  -26.724964  -26.427097  ...  -6.857521   -8.794199
    -6.7176   ]
  ...
  [ 13.886283  -26.630348  -26.375664  ...  -5.5347486  -8.12678
    -6.620142 ]
  [ 13.886637  -26.630585  -26.375904  ...  -5.53475    -8.126849
    -6.6202025]
  [ 13.66938   -27.155464  -26.877563  ...  -6.889975   -9.146458
    -6.1848373]]], shape=(1, 146, 32), dtype=float32)
TIME TAKEN: 0.8099558353424072


In [49]:
import numpy as np
np.allclose(onnx_outputs, outputs.numpy(), atol=1e-2)

True